# Exercise 1.5 - Histograms and Colorbars
prepared by M.Hauser

In this exercise we will look at how to create histograms and get to know a bit more about colorbars.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import xarray as xr

%matplotlib inline

## Create/ load data

#### artificial data

First, we create some artificial multivariate normal data.

In [ ]:
mean = [0, 0]
cov = [[1, 1], [1, 2]]
X0, Y0 = np.random.multivariate_normal(mean, cov, 10000).T

cov = [[1, 0], [0, 1]]  # diagonal covariance
X1, Y1 = np.random.multivariate_normal(mean, cov, 10000).T

#### Swiss Station Data

We use Station Data for Switzerland - the time series of temperature & precipitation.

The data is avaliable from [MeteoSwiss](http://www.meteoswiss.admin.ch/home/climate/past/homogenous-monthly-data.html).

The data has already been [retrieved and postprocessed](../data/prepare_data_MCH.ipynb).

In [ ]:
def load_mch(station, annual=True):
    fN = '../data/MCH_HOM_{}.nc'.format(station)
    return xr.open_dataset(fN)
    
BAS = load_mch('BAS')
DAV = load_mch('DAV')
SAE = load_mch('SAE')
GSB = load_mch('GSB')

## Histogram

It's quite easy to create a histogram:

In [ ]:
plt.hist(X0);

`hist` takes many parameters:

In [ ]:
plt.hist(Y0,
         bins=30,               # number of bins
         density=True,          # percentage instead of count
         histtype='stepfilled', # don't apply the edgecolor to the individual bars
         facecolor='indianred',
         edgecolor='0.1');

### Exercise

 * Plot the distribution of precipitation in Basel.

In [ ]:
# plot here
x = BAS.Precipitation


### Solution

In [ ]:
x = BAS.Precipitation
plt.hist(x, bins=20);

### Exercise

 * Plot the distribution of temperature in Basel and Davos in the same axes.
 * Use the `alpha` keyword to make them semi-transparent.

In [ ]:
# code here
f, ax = plt.subplots()


### Solution

In [ ]:
# code here
f, ax = plt.subplots()

ax.hist(BAS.Temperature, bins=20, alpha=0.5)
ax.hist(DAV.Temperature, bins=20, alpha=0.5);

## 2D histograms and hexbin

We can also do 2D histograms with `hist2d` and hexagonal binning plots with `hexbin` by passing x and y.

In [ ]:
f, axes = plt.subplots(1, 2)
f.set_size_inches(20 / 2.54, 10 / 2.54)

ax = axes[0]
ax.hist2d(X0, Y0, bins=25, cmap='Reds')

ax = axes[1]
ax.hexbin(X1, Y1, gridsize=20, cmap='Blues')

### Exercise

* use the data for Basel (`BAS`) and Col du Grand St-Bernard (`GSB`)

* Is there a correlation between temperatures? 
  * Use hist2d to find out
* Is there a correlation between precipitation? 
  * Use hexbin to find out   

In [ ]:
f, axes = plt.subplots(1, 2)
f.set_size_inches(20 / 2.54, 10 / 2.54)

# ======
x = GSB.Temperature
y = BAS.Temperature

ax = axes[0]


# ======
x = GSB.Precipitation.values
y = BAS.Precipitation.values

ax = axes[1]



### Solution

In [ ]:
f, axes = plt.subplots(1, 2)
f.set_size_inches(20 / 2.54, 10 / 2.54)

# ======
x = GSB.Temperature
y = BAS.Temperature

ax = axes[0]
ax.hist2d(x, y, bins=25, cmap='Reds')

# ======
x = GSB.Precipitation.values
y = BAS.Precipitation.values

ax = axes[1]
ax.hexbin(x, y, cmap='Blues', gridsize=20);

### Solution if you get an error (else you can ignore this)

Matplolib can only handle NaNs in hexbin since version 2.2. As precipitation at GSB contains NaNs, we may need to get rid of them.

In [ ]:
f, axes = plt.subplots(1, 2)
f.set_size_inches(20 / 2.54, 10 / 2.54)

# ======
x = GSB.Temperature
y = BAS.Temperature

ax = axes[0]

ax.hist2d(x, y, bins=25, cmap='Reds')

# ======
x = GSB.Precipitation.values
y = BAS.Precipitation.values

# find all values that are NaN
sel = ~(np.isnan(x) | np.isnan(y))

# drop them
x = x[sel]
y = y[sel]

ax = axes[1]

ax.hexbin(x, y, cmap='Blues', gridsize=20);

While the plots look fine, we don't know how many elements there are in each bin - we need colorbars!


## colorbars 

We got to know colorbars in exercise 1.3. They are created with `plt.colorbar(h)`

A colorbar is an `axes` object with some special properties. Therefore they are a function of `plt` (or of `f`), but not of `ax`. They are vertical per default, but we can make them horizontal with the `orientation` keyword. Also adding a label is easy. It has a `set_label` function that adds a x- or y- label, depending on the orientation.

We need to pass a `mappable` to the colorbar. A `mappable` is returned by almost all plotting function. I usually call them `h`.

In [ ]:
f, ax = plt.subplots(1, 1)

h = ax.hexbin(X0, Y0, cmap='Blues', gridsize=20);

cbar = f.colorbar(h)
cbar.set_label('counts per bin')

### Exercise
 * add a horizontal colorbar

In [ ]:
f, ax = plt.subplots(1, 1)

h = ax.hexbin(GSB.Temperature, BAS.Temperature, cmap='Reds', gridsize=20);

# add colorbar


### Solution

In [ ]:
f, ax = plt.subplots(1, 1)

h = ax.hexbin(GSB.Temperature, BAS.Temperature, cmap='Reds', gridsize=20);

# add colorbar

cbar = f.colorbar(h, orientation='horizontal')
cbar.set_label('counts per bin')

## Specifying the axes the colorbar belongs to

The `ax` keyword in `colorbar` determines which axes the colorbar belongs to. You can even pass a list of axes, to get a colorbar that spans multiple subfigures.

> this shrinks the axes to make room for the colorbar.

In [ ]:
f, axes = plt.subplots(2, 2)
axes = axes.flatten()
f.subplots_adjust(wspace=0.4)

ax = axes[0]
h0 = ax.hexbin(X0, Y0, cmap='viridis', gridsize=20);

ax = axes[1]
h2 = ax.hexbin(X1, Y1, cmap='Reds', gridsize=20, vmax=250);

ax = axes[2]
h1 = ax.hexbin(X1, Y1, cmap='Blues', gridsize=20);

ax = axes[3]
h2 = ax.hexbin(X0, Y0, cmap='Reds', gridsize=20, vmax=250);


# ======
# single colorbars

f.colorbar(h0, ax=axes[0])
f.colorbar(h1, ax=axes[2])

# ======
# double colorbars

cbar = f.colorbar(h2, ax=[axes[1], axes[3]])
cbar.set_label('Double colorbar')



> You have to make sure that all plots with the same colorbar also display the same data range! You do that by setting `vmin` and `vmax`.

### Exercise

Let's go back to the figure comparing BAS and GSB.

 * add a colorbar spanning both axes
 * make sure both `hexbin`s show the same scale (30 is a good maximum value)

In [ ]:
x0 = GSB.Temperature
y0 = BAS.Temperature

x1 = GSB.Precipitation
y1 = BAS.Precipitation

# ======

f, axes = plt.subplots(1, 2)
f.set_size_inches(20 / 2.54, 10 / 2.54)

# ======

ax = axes[0]
h = ax.hexbin(x0, y0, cmap='Blues', gridsize=25, vmax=30);

# ======

ax = axes[1]
h = ax.hexbin(x1, y1, cmap='Blues', gridsize=25, vmax=30);

# ======
# add colorbar



### Solution

In [ ]:
x0 = GSB.Temperature
y0 = BAS.Temperature

x1 = GSB.Precipitation
y1 = BAS.Precipitation

# ======

f, axes = plt.subplots(1, 2)
f.set_size_inches(20 / 2.54, 10 / 2.54)

# ======

ax = axes[0]
h = ax.hexbin(x0, y0, cmap='Blues', gridsize=25, vmax=30);

# ======

ax = axes[1]
h = ax.hexbin(x1, y1, cmap='Blues', gridsize=25, vmax=30);

# ======
# add colorbar

f.colorbar(h, ax=[axes[0], axes[1]], orientation='horizontal')

Actually, that does not look great, using only one vertical colorbar is better - let's do that.

### Bonus exercise

* add only one vertical colorbar for the right axes
  * remove the old colorbar
  * add a label (`'counts per bin'`)
* add x-and y-labels
* add titles



In [ ]:
x0 = GSB.Temperature
y0 = BAS.Temperature

x1 = GSB.Precipitation
y1 = BAS.Precipitation

# ======

f, axes = plt.subplots(1, 2)
f.set_size_inches(20 / 2.54, 10 / 2.54)

# ======

ax = axes[0]
h = ax.hexbin(x0, y0, cmap='Blues', gridsize=25, vmax=30);

# add labels and title

# ======

ax = axes[1]
h = ax.hexbin(x1, y1, cmap='Blues', gridsize=25, vmax=30);

# add labels and title


# ======
# add colorbar


plt.tight_layout()

### Solution

In [ ]:
x0 = GSB.Temperature
y0 = BAS.Temperature

x1 = GSB.Precipitation
y1 = BAS.Precipitation

# ======

f, axes = plt.subplots(1, 2)
f.set_size_inches(20 / 2.54, 10 / 2.54)

# ======

ax = axes[0]
h = ax.hexbin(x0, y0, cmap='Blues', gridsize=25, vmax=30);

# add labels and title
ax.set_title('Temperature')
ax.set_title('a', loc='left', weight='semibold')

ax.set_xlabel('GSB [°C]')
ax.set_ylabel('BAS [°C]')


# ======

ax = axes[1]
h = ax.hexbin(x1, y1, cmap='Blues', gridsize=25, vmax=30);

# add labels and title
ax.set_title('Precipitation')
ax.set_title('b', loc='left', weight='bold')

ax.set_xlabel('GSB [mm / yr]')
ax.set_ylabel('BAS [mm / yr]')

# ======
# add colorbar

cbar = f.colorbar(h, ax=ax)
cbar.set_label('counts per bin')

plt.tight_layout()